<a href="https://colab.research.google.com/github/peckert659/course_app/blob/main/best_trajet_cff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
#!/usr/bin/env python3
"""
fetch_cff.py — Trouve le meilleur trajet CFF/SBB pour une heure d'arrivée souhaitée.

Utilisation :
    python fetch_cff.py --from "Lausanne" --to "Genève" --arrival "2025-07-10 15:30"

Arguments :
  --from      (obligatoire)  Station de départ
  --to        (obligatoire)  Station d'arrivée
  --arrival   (obligatoire)  Date et heure d'arrivée souhaitée au format « YYYY-MM-DD HH:MM » (heure locale Zurich)
  --limit     (optionnel)    Nombre maximum de trajets à interroger (défaut : 8)

Le script utilise l'API REST non-officielle transport.opendata.ch :
  https://transport.opendata.ch/v1/connections
(on passe le paramètre isArrivalTime=1 pour indiquer que l'heure fournie est une heure d'arrivée).

Logique de sélection :
  1. Récupération jusqu'à « --limit » trajets arrivant autour de l'heure cible.
  2. Filtrage des trajets qui arrivent à l'heure souhaitée ou avant.
  3. Choix du trajet partant le plus tard (mais toujours arrivant à temps).
  4. En cas d'égalité sur l'heure de départ, on privilégie la durée la plus courte ;
     les durées qui diffèrent de ±10 minutes sont considérées identiques.
  5. Si les durées sont équivalentes, on garde le premier trajet retourné.

Sortie :
  - Heure et quai de départ
  - Heure et quai d'arrivée
  - Durée totale
  - Nombre de correspondances
  - Détail de chaque segment (train, bus, marche, etc.)
  - Liste des trajets disponibles
"""

import argparse
import requests
import sys
from datetime import datetime, timedelta
from typing import List, Dict, Any
from zoneinfo import ZoneInfo

API_URL = "https://transport.opendata.ch/v1/connections"


def parse_args() -> argparse.Namespace:
    parser = argparse.ArgumentParser(description="Cherche le meilleur trajet CFF/SBB")
    parser.add_argument("--from", dest="origin", required=True, help="Station de départ")
    parser.add_argument("--to", dest="destination", required=True, help="Station d'arrivée")
    parser.add_argument("--arrival", required=True, help="Date et heure d'arrivée (YYYY-MM-DD HH:MM, heure locale Zurich)")
    parser.add_argument("--limit", type=int, default=8, help="Nombre maximum de trajets à récupérer (défaut : 8)")
    return parser.parse_args()


def parse_duration(duration_str: str) -> timedelta:
    days = 0
    if "d" in duration_str:
        days_part, duration_str = duration_str.split("d", 1)
        days = int(days_part)
    hours, minutes, seconds = map(int, duration_str.split(":"))
    return timedelta(days=days, hours=hours, minutes=minutes, seconds=seconds)


def fetch_connections(origin: str, destination: str, arrival_dt: datetime, limit: int = 8) -> List[Dict[str, Any]]:
    # Ajouter une heure pour contourner la limitation de l'API
    adjusted_arrival = arrival_dt + timedelta(hours=1)
    params = {
        "from": origin,
        "to": destination,
        "date": adjusted_arrival.strftime("%Y-%m-%d"),
        "time": adjusted_arrival.strftime("%H:%M"),
        "isArrivalTime": 1,
        "limit": limit,
    }
    response = requests.get(API_URL, params=params, timeout=10)
    response.raise_for_status()
    data = response.json()
    return data.get("connections", [])


def choose_best_connection(connections: List[Dict[str, Any]], target_arrival: datetime) -> Dict[str, Any]:
    best = None
    latest_departure = None
    best_dur = None

    for conn in connections:
        arr_iso = conn["to"]["arrival"]
        dep_iso = conn["from"]["departure"]
        if arr_iso is None or dep_iso is None:
            continue

        arr_time = datetime.fromisoformat(arr_iso)
        dep_time = datetime.fromisoformat(dep_iso)

        if arr_time > target_arrival:
            continue

        dur = parse_duration(conn["duration"])

        if best is None:
            best, latest_departure, best_dur = conn, dep_time, dur
            continue

        if dep_time > latest_departure:
            best, latest_departure, best_dur = conn, dep_time, dur
            continue

        if dep_time == latest_departure:
            if abs(dur - best_dur) <= timedelta(minutes=10):
                continue
            if dur < best_dur:
                best, latest_departure, best_dur = conn, dep_time, dur

    return best


def print_connection(conn: Dict[str, Any]) -> None:
    from_stop = conn["from"]
    to_stop = conn["to"]
    duration_td = parse_duration(conn["duration"])

    print("\n=== Trajet sélectionné ===")
    print(f"Départ  : {from_stop['station']['name']} · {from_stop['departure']} · quai {from_stop.get('platform', '')}")
    print(f"Arrivée : {to_stop['station']['name']} · {to_stop['arrival']} · quai {to_stop.get('platform', '')}")
    print(f"Durée   : {duration_td}\nTransferts : {conn['transfers']}")

    print("\n--- Détails par segment ---")
    for idx, section in enumerate(conn["sections"], start=1):
        dep = section["departure"]
        arr = section["arrival"]
        journey = section.get("journey")

        if journey:
            print(f"{idx}. {journey['category']} {journey['number']} | {dep['station']['name']} ({dep['departure'][11:16]}) -> {arr['station']['name']} ({arr['arrival'][11:16]})")
        else:
            distance = section.get("walk", {}).get("distance", "?")
            print(f"{idx}. Marche (~{distance} m) | {dep['station']['name']} -> {arr['station']['name']}")


def print_all_connections(connections: List[Dict[str, Any]]) -> None:
    print("\n=== Tous les trajets disponibles ===")
    for idx, conn in enumerate(connections, start=1):
        from_stop = conn["from"]
        to_stop = conn["to"]
        duration_td = parse_duration(conn["duration"])
        print(f"{idx}. {from_stop['departure']} -> {to_stop['arrival']} | Durée : {duration_td} | Transferts : {conn['transfers']}")


def run_fetch_cff_colab(origin: str, destination: str, arrival_str: str, limit: int = 8):
    try:
        target_arrival = datetime.strptime(arrival_str, "%Y-%m-%d %H:%M").replace(tzinfo=ZoneInfo("Europe/Zurich"))
    except ValueError:
        print("Format d'arrivée invalide. Utilisez YYYY-MM-DD HH:MM")
        return

    try:
        connections = fetch_connections(origin, destination, target_arrival, limit)
    except requests.RequestException as e:
        print(f"Erreur lors de la requête API : {e}")
        return

    if not connections:
        print("Aucun trajet trouvé ✖️")
        return

    print_all_connections(connections)
    best_conn = choose_best_connection(connections, target_arrival)
    print_connection(best_conn)


def main() -> None:
    args = parse_args()
    try:
        target_arrival = datetime.strptime(args.arrival, "%Y-%m-%d %H:%M").replace(tzinfo=ZoneInfo("Europe/Zurich"))
    except ValueError:
        sys.exit("Format d'arrivée invalide. Utilisez YYYY-MM-DD HH:MM")

    try:
        connections = fetch_connections(args.origin, args.destination, target_arrival, args.limit)
    except requests.RequestException as e:
        sys.exit(f"Erreur lors de la requête API : {e}")

    if not connections:
        sys.exit("Aucun trajet trouvé ✖️")

    best_conn = choose_best_connection(connections, target_arrival)
    print_all_connections(connections)
    print_connection(best_conn)





In [34]:
run_fetch_cff_colab("Montreux", "Ovronnaz", "2025-07-10 09:00", limit=12)




=== Tous les trajets disponibles ===
1. 2025-07-09T07:36:00+0200 -> 2025-07-09T09:48:00+0200 | Durée : 2:12:00 | Transferts : 2
2. 2025-07-09T09:10:00+0200 -> 2025-07-09T10:42:00+0200 | Durée : 1:32:00 | Transferts : 2
3. 2025-07-09T10:36:00+0200 -> 2025-07-09T12:13:00+0200 | Durée : 1:37:00 | Transferts : 2
4. 2025-07-09T11:10:00+0200 -> 2025-07-09T12:53:00+0200 | Durée : 1:43:00 | Transferts : 2
5. 2025-07-09T12:36:00+0200 -> 2025-07-09T14:26:00+0200 | Durée : 1:50:00 | Transferts : 2
6. 2025-07-09T15:10:00+0200 -> 2025-07-09T16:48:00+0200 | Durée : 1:38:00 | Transferts : 2
7. 2025-07-09T16:10:00+0200 -> 2025-07-09T18:04:00+0200 | Durée : 1:54:00 | Transferts : 2
8. 2025-07-09T18:10:00+0200 -> 2025-07-09T19:49:00+0200 | Durée : 1:39:00 | Transferts : 2
9. 2025-07-10T00:39:00+0200 -> 2025-07-10T06:34:00+0200 | Durée : 5:55:00 | Transferts : 3
10. 2025-07-10T05:26:00+0200 -> 2025-07-10T07:19:00+0200 | Durée : 1:53:00 | Transferts : 2
11. 2025-07-10T06:36:00+0200 -> 2025-07-10T08:19:00